<a href="https://colab.research.google.com/github/Abhijitgayen/SNA_As_Msc_projects/blob/main/main_work/LSTM%2BW2V%2BsemEval_2016_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ## Libary_enter

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd

import tensorflow_hub as hub
import numpy as np

In [2]:
# Load Pretrained Word2Vec
embed = hub.load("https://tfhub.dev/google/Wiki-words-250/2")

## Some_function for prepossing

In [3]:
def get_max_length(df):
    """
    get max token counts from train data, 
    so we use this number as fixed length input to RNN cell
    """
    max_length = 0
    for row in df['tweet']:
        if len(row.split(" ")) > max_length:
            max_length = len(row.split(" "))
    return max_length

def get_word2vec_enc(reviews):
    """
    get word2vec value for each word in sentence.
    concatenate word in numpy array, so we can use it as RNN input
    """
    encoded_reviews = []
    for review in reviews:
        tokens = review.split(" ")
        word2vec_embedding = embed(tokens)
        encoded_reviews.append(word2vec_embedding)
    return encoded_reviews
        
def get_padded_encoded_reviews(encoded_reviews):
    """
    for short sentences, we prepend zero padding so all input to RNN has same length
    """
    padded_reviews_encoding = []
    for enc_review in encoded_reviews:
        zero_padding_cnt = max_length - enc_review.shape[0]
        pad = np.zeros((1, 250))
        for i in range(zero_padding_cnt):
            enc_review = np.concatenate((pad, enc_review), axis=0)
        padded_reviews_encoding.append(enc_review)
    return padded_reviews_encoding

def sentiment_encode(sentiment):
    """
    return one hot encoding for Y value
    """
    if sentiment == 'positive':
        return [1,0]
    if sentiment == 'neutral':
        return [0.5,0.5]
    else:
        return [0,1]
    
def preprocess(df):
    """
    encode text value to numeric value
    """
    # encode words into word2vec
    reviews = df['tweet'].tolist()
    
    encoded_reviews = get_word2vec_enc(reviews)
    padded_encoded_reviews = get_padded_encoded_reviews(encoded_reviews)
    # encoded sentiment
    sentiments = df['label'].tolist()
    encoded_sentiment = [sentiment_encode(sentiment) for sentiment in sentiments]
    X = np.array(padded_encoded_reviews)
    Y = np.array(encoded_sentiment)
    return X, Y 

## Data_Set_add

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
tweetsDatas=pd.read_csv('/content/drive/MyDrive/2017_data_set/dataset/train/twitter-2016dev-A.txt', sep="\t", header=None, names=["id","label", "tweet"]); #skiping these two rows as they have some bad data
testData=pd.read_csv('/content/drive/MyDrive/2017_data_set/dataset/train/twitter-2016devtest-A.txt',sep="\t", header=None, names=["id","label", "tweet"]); #test_data

In [10]:
print(tweetsDatas.head())
print(tweetsDatas.label.value_counts())
print(testData.head())
print(testData.label.value_counts())

                   id     label  \
0  638060586258038784   neutral   
1  638061181823922176  positive   
2  638083821364244480   neutral   
3  638091450132078593  positive   
4  638125563790557184  positive   

                                               tweet  
0  05 Beat it - Michael Jackson - Thriller (25th ...  
1  Jay Z joins Instagram with nostalgic tribute t...  
2  Michael Jackson: Bad 25th Anniversary Edition ...  
3  I liked a @YouTube video http://t.co/AaR3pjp2P...  
4  18th anniv of Princess Diana's death. I still ...  
positive    829
neutral     746
negative    391
Name: label, dtype: int64
                   id     label  \
0  637641175948763136   neutral   
1  637651487762554881   neutral   
2  637666734300905472  negative   
3  637668142110654468   neutral   
4  637708370129125377  positive   

                                               tweet  
0  @SeeMonterey LOST - Sony cell phone with holid...  
1  @PersonaSoda well yeah, that's third parties. ...  
2  Sony r

In [6]:
df=tweetsDatas
# max_length is used for max sequence of input
df=df.drop(1829,axis='rows')
max_length = get_max_length(df)
print(max_length)

train_X, train_Y = preprocess(df)

31


## ALL_models

In [7]:
# LSTM model
model = Sequential()
model.add(LSTM(128))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# LSTM model
model = Sequential()
model.add(LSTM(128))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

## train Models

In [8]:
print('Train...')
model.fit(train_X, train_Y,epochs=100)

Train...
Epoch 1/100
62/62 [==============================] - 6s 58ms/step - loss: 0.6332 - accuracy: 0.7654
Epoch 2/100
62/62 [==============================] - 4s 57ms/step - loss: 0.6025 - accuracy: 0.7623
Epoch 3/100
62/62 [==============================] - 4s 59ms/step - loss: 0.5928 - accuracy: 0.7634
Epoch 4/100
62/62 [==============================] - 4s 58ms/step - loss: 0.5775 - accuracy: 0.7644
Epoch 5/100
62/62 [==============================] - 4s 60ms/step - loss: 0.5664 - accuracy: 0.7710
Epoch 6/100
62/62 [==============================] - 5s 79ms/step - loss: 0.5656 - accuracy: 0.7573
Epoch 7/100
62/62 [==============================] - 4s 62ms/step - loss: 0.5535 - accuracy: 0.7567
Epoch 8/100
62/62 [==============================] - 4s 72ms/step - loss: 0.5512 - accuracy: 0.7690
Epoch 9/100
62/62 [==============================] - 5s 77ms/step - loss: 0.5392 - accuracy: 0.7649
Epoch 10/100
62/62 [==============================] - 4s 70ms/step - loss: 0.5348 - accurac

In [9]:
test_df=testData
print(test_df.label.value_counts())
max_length = get_max_length(test_df)
test_X, test_Y = preprocess(test_df)

score, acc = model.evaluate(test_X, test_Y, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

positive    994
neutral     681
negative    325
Name: label, dtype: int64
63/63 - 2s - loss: 1.1565 - accuracy: 0.6800 - 2s/epoch - 34ms/step
Test score: 1.1564884185791016
Test accuracy: 0.6800000071525574
